7.4.2 im2col로 데이터 전개하기

In [ ]:
# im2col은 입력 데이터를 필터링하기 좋게 전개하는 함수

In [1]:
import numpy as np

In [2]:
x,y = 0,0
x_max, y_max = 2,2
stride = 1

In [7]:
 col = np.zeros((2,3,3,3,2,2))

In [4]:
img = np.zeros((2,3,4,4))

In [5]:
for i in range(2):
    for j in range(3):
        img[i][j] = np.array([
            [0,1,2,3],
            [4,5,6,7],
            [8,9,10,11],
            [12,13,14,15]
        ])
        img[i][j] += 48*i+16*j
img

array([[[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]],

        [[32., 33., 34., 35.],
         [36., 37., 38., 39.],
         [40., 41., 42., 43.],
         [44., 45., 46., 47.]]],


       [[[48., 49., 50., 51.],
         [52., 53., 54., 55.],
         [56., 57., 58., 59.],
         [60., 61., 62., 63.]],

        [[64., 65., 66., 67.],
         [68., 69., 70., 71.],
         [72., 73., 74., 75.],
         [76., 77., 78., 79.]],

        [[80., 81., 82., 83.],
         [84., 85., 86., 87.],
         [88., 89., 90., 91.],
         [92., 93., 94., 95.]]]])

In [16]:

for y in range(3):
        y_max = y + stride*2
        for x in range(3):
            x_max = x + stride*2
            print("x,y:",img[:, :, y:y_max:stride, x:x_max:stride])
            print(col[:,:,y,x,:,:].shape)
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
#col

x,y: [[[[ 0.  1.]
   [ 4.  5.]]

  [[16. 17.]
   [20. 21.]]

  [[32. 33.]
   [36. 37.]]]


 [[[48. 49.]
   [52. 53.]]

  [[64. 65.]
   [68. 69.]]

  [[80. 81.]
   [84. 85.]]]]
(2, 3, 2, 2)
(2, 3, 2, 2)
x,y: [[[[ 1.  2.]
   [ 5.  6.]]

  [[17. 18.]
   [21. 22.]]

  [[33. 34.]
   [37. 38.]]]


 [[[49. 50.]
   [53. 54.]]

  [[65. 66.]
   [69. 70.]]

  [[81. 82.]
   [85. 86.]]]]
(2, 3, 2, 2)
(2, 3, 2, 2)
x,y: [[[[ 2.  3.]
   [ 6.  7.]]

  [[18. 19.]
   [22. 23.]]

  [[34. 35.]
   [38. 39.]]]


 [[[50. 51.]
   [54. 55.]]

  [[66. 67.]
   [70. 71.]]

  [[82. 83.]
   [86. 87.]]]]
(2, 3, 2, 2)
(2, 3, 2, 2)
x,y: [[[[ 4.  5.]
   [ 8.  9.]]

  [[20. 21.]
   [24. 25.]]

  [[36. 37.]
   [40. 41.]]]


 [[[52. 53.]
   [56. 57.]]

  [[68. 69.]
   [72. 73.]]

  [[84. 85.]
   [88. 89.]]]]
(2, 3, 2, 2)
(2, 3, 2, 2)
x,y: [[[[ 5.  6.]
   [ 9. 10.]]

  [[21. 22.]
   [25. 26.]]

  [[37. 38.]
   [41. 42.]]]


 [[[53. 54.]
   [57. 58.]]

  [[69. 70.]
   [73. 74.]]

  [[85. 86.]
   [89. 90.]]]]
(2, 3, 2, 2)
(2,

7.4.3 합성곱 계층 구현하기

In [17]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [18]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)
# 여기서 out_h, out_w = (7+ 2*0 - 5)/1 + 1 = 3이므로 
# (1*out_h*out_w, channel*filter_w*filter_h) = (9, 75)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [20]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad 

    def forward(self, x):
        # # of filters, channel, filter height, filter width
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        # col_W = self.W.reshape(1,-1) 과 같지 않을까? 
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)

        return out 


7.4.4 풀링 계층 구현하기

In [ ]:
# 채널마다 im2col를 독립적으로 전개 후, 행별 max값을 구하고 다시 형상을 바꿔줌

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N,C,H,W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # im2col 함수로 2차원 배열로 만들어줌 
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        # 이 때의 col.shape = (N*OH*OW, C*PH*PW)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        # 이 때의 col.shape = (N*OH*OW*C, PH*PW)

        out = np.max(col, axis=1)
        # col.shape = (N*OH*OW*C, PH*PW) 이므로
        # 이에 맞게 reshape() 해준 뒤 (N,C,OH,OW) 형태로 transpose 해줌 
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

        return out 

7.5 CNN 구현하기

In [ ]:
# simple_convnet.py
# train_convnet.py 참고 

7.6.1 1번째 층의 가중치 시각화하기 

In [ ]:
# visualize_filter.py 참고 